## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-10-23-45-15 +0000,nypost,Atlanta deputy charged with sending nude pictu...,https://nypost.com/2026/01/10/us-news/atlanta-...
1,2026-01-10-23-41-23 +0000,wapo,Thousands protest ICE presence in Minneapolis ...,https://www.washingtonpost.com/nation/2026/01/...
2,2026-01-10-23-39-24 +0000,nyt,U.S. Launches Major Strikes on Islamic State T...,https://www.nytimes.com/2026/01/10/us/politics...
3,2026-01-10-23-31-57 +0000,latimes,Actor T.K. Carter dies at 69,https://www.latimes.com/california/story/2026-...
4,2026-01-10-23-20-56 +0000,nypost,Fallen paralyzed NYC Det. Steven McDonald hono...,https://nypost.com/2026/01/10/us-news/paralyze...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2405/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
179,trump,45
10,ice,22
65,venezuela,19
38,iran,19
153,protests,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
204,2026-01-10-00-23-02 +0000,nypost,President Trump wears ‘Happy Trump’ lapel pin ...,https://nypost.com/2026/01/09/us-news/presiden...,114
23,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...,93
202,2026-01-10-00-36-00 +0000,wsj,Iran’s Supreme Leader Says He Won’t Bow Down t...,https://www.wsj.com/world/middle-east/irans-su...,92
53,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,90
95,2026-01-10-15-19-51 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
204,114,2026-01-10-00-23-02 +0000,nypost,President Trump wears ‘Happy Trump’ lapel pin ...,https://nypost.com/2026/01/09/us-news/presiden...
23,93,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...
209,57,2026-01-10-00-05-41 +0000,bbc,South Africa's strained ties with US face new ...,https://www.bbc.com/news/articles/c62wxezynk2o...
113,40,2026-01-10-13-00-00 +0000,nypost,DSA leaders paint Venezuela as socialist utopi...,https://nypost.com/2026/01/10/us-news/dsa-lead...
211,39,2026-01-10-00-01-00 +0000,wsj,Wall Street’s move after the 2008 housing cras...,https://www.wsj.com/economy/housing/trump-blin...
75,34,2026-01-10-17-46-22 +0000,cbc,Rescuers detect 'signs of life' after garbage ...,https://www.cbc.ca/news/world/philippines-garb...
104,34,2026-01-10-14-18-09 +0000,nypost,Iran activates ‘missile cities’ as protests ra...,https://nypost.com/2026/01/10/world-news/iran-...
41,32,2026-01-10-20-53-00 +0000,wsj,The Defense Department has selected Owen West ...,https://www.wsj.com/politics/national-security...
177,32,2026-01-10-02-53-01 +0000,nypost,"Trump calls for a 10%, one-year cap on credit ...",https://nypost.com/2026/01/09/us-news/trump-ca...
94,30,2026-01-10-15-31-39 +0000,nypost,Russian attacks devastate Ukraine’s power grid...,https://nypost.com/2026/01/10/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
